Ricardo Pineda | 20160164

# Tarea resolución de Crimen

Se ha cometido un delito y la policía necesita su ayuda. Un detective te dio el informe de la
escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue
un asesinato que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL
City. Empiece por recuperar el informe de la escena del crimen correspondiente de la base de
datos del departamento de policía

### Imports

In [16]:
import pandas as pd
import sqlite3
import numpy as np
np.set_printoptions(threshold=np.inf)

### Conexión

In [5]:
conn = sqlite3.connect("sql-murder-mystery.db")

In [8]:
init_table = pd.read_sql("SELECT * FROM sqlite_master;", conn)
init_table.head()

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,CREATE TABLE crime_scene_report (\n dat...
1,table,drivers_license,drivers_license,3,CREATE TABLE drivers_license (\n id int...
2,table,person,person,4,CREATE TABLE person (\n id integer PRIM...
3,table,facebook_event_checkin,facebook_event_checkin,5,CREATE TABLE facebook_event_checkin (\n ...
4,table,interview,interview,6,CREATE TABLE interview (\n person_id in...


In [10]:
crime_table = pd.read_sql("SELECT * FROM crime_scene_report;", conn)
crime_table.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


### Filtros

 Sabemos algunos detalles del asesinato: 
 
 - Ocurrio el 15 de enero del 2018
 
 - Ocurrio en SQL City

In [11]:
filtered_table = pd.read_sql("""SELECT * FROM crime_scene_report 
                WHERE type = 'murder'
                AND date = 20180115
                AND city = 'SQL City';""", conn)
filtered_table.head()

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


Descripcion del asesinato de ese dia en SQL City

In [27]:
filtered_table['description'][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

#### Datos importantes: 

2 Testigos:

- El primer testigo vive en Northwestern Dr, no sabemos su nombre, pero si que vive en la ultima casa.

- El segundo testigo se llama Annabel y vive en Franklin Ave.

### Testigos

In [38]:
primer_testigo = pd.read_sql("""SELECT * FROM person
                        WHERE address_street_name = 'Northwestern Dr' ORDER BY address_number DESC;""", conn) #Sabemos que vive en la ultima casa
primer_testigo.head(1)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


El primer testigo se llama Morty Schapiro, licencia #118009

In [50]:
segundo_testigo = pd.read_sql("""SELECT * FROM person WHERE name LIKE '%Annabel%'
                        and address_street_name LIKE '%Franklin Ave%';""", conn)
segundo_testigo

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Ahora sabemos que su numbre completo es Annabel Miller, licencia #490173

Hay una base de datos que contiene las declaraciones de los testigos, veamos

In [58]:
declaraciones = pd.read_sql("""SELECT * FROM interview""", conn)
declaraciones.head(1)

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n


Solo se identifica por ID, no por nombre. Afortunadamente tenemos el ID de ambos testigos:

- Morty = 14887

- Annabel = 16371

### Declaraciones

In [54]:
dec_testigo_uno = pd.read_sql("SELECT * FROM interview WHERE person_id = 14887;", conn) # Morty
dec_testigo_dos =pd.read_sql("SELECT * FROM interview WHERE person_id = 16371;", conn) # Annabel

### Declaraciones de Morty:

In [61]:
dec_testigo_uno["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

### Declaraciones de Annabel

In [62]:
dec_testigo_dos["transcript"][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

## Datos de interes:

Ambos testigos hablan de un gimnasio. Segun Morty el sospechoso tenia una bolsa que leia: "Get Fit Now Gym", por lo que podemos suponer que ese es el nombre del gimnasio. 

Ademas, señala que el numero de membresia impreso en la bolsa empezaba con "48Z", y que solo los miembros de oro tienen esas bolsas.

La placa del auto del sospechoso empieza con "H42W"

##### Gimnasio

Tenemos acceso a la base de datos del gimnasio, por lo que podemos ver a los miembros que tengan una membresia que empiece con "48Z" y que sean de membresia oro

In [67]:
gimnasio = pd.read_sql("SELECT * FROM get_fit_now_member WHERE membership_status = 'gold' AND id LIKE '%48Z%';", conn)
gimnasio

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Nos devuelve a dos sospechosos que cumplen con los criterios. Podria ser cualquiera.

##### Licencia

Ya que tenemos una base de datos que incluye las licencias de conducir, podemos buscar en ella si alguna coincide con las declaraciones del testigo

In [65]:
licencia = pd.read_sql("SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%' AND gender = 'male';", conn)
licencia

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


Obtenemos los datos de dos vehiculos que cumplen con las caracetristicas. De nuevo, podria ser cualquiera.

### Atando los hilos...

Hemos reducido la busqueda a un total de dos personas, pero tenemos un detalle mas; Annabel nos dijo que habia visto al sospechoso el 09 de enero del 2018. Que pasa si buscamos los datos de ese dia?

In [68]:
pd.read_sql("""SELECT * FROM get_fit_now_check_in
            WHERE check_in_date = 20180109 and membership_id = '48Z55';""", conn)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z55,20180109,1530,1700


Podemos ver que ese dia solo fue una persona al gimnasio. Id de membresia = 48Z55

### Culpable

In [69]:
culpable = pd.read_sql("SELECT * FROM get_fit_now_member WHERE id = '48Z55';", conn)
culpable

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


Viendo el Id de su membresia, podemos concluir que el culpable es "Jeremy Bowers"

### Comprobacion:

In [73]:
cursor = conn.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

Encontramos al culpable del crimen, ahora hace falta encontrar a la mente maestra detras de todo

### Master mind

Analicemos las declaraciones del asesino. Su ID es 67318

In [74]:
dec_asesino = pd.read_sql("SELECT * FROM interview WHERE person_id = 67318;", conn)
dec_asesino['transcript'][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

Tenemos varios detalles de quien esta detras realmente:

- Es una mujer adinerada
 
- Estatura entre 5'5 y 5'7

- Tiene un Tesla Model S

- Fue a la sinfonia SQL 3 veces solo en diciembre del 2017

Metamoslo todo dentro de un query

In [75]:
sospechosos1 = pd.read_sql("""SELECT * FROM drivers_license
                JOIN person ON person.license_id = drivers_license.id 
                WHERE car_make LIKE '%Tesla%' AND car_model = 'Model S'
                AND hair_color = 'red' 
                AND height BETWEEN 65 and 67 AND gender = 'female';""", conn)
sospechosos1

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,918773,48,65,black,red,female,917UU3,Tesla,Model S,78881,Red Korb,918773,107,Camerata Dr,961388910
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S,90700,Regina George,291182,332,Maple Ave,337169072
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


Hay tres personas que cumplen con los criterios, podria ser cualquiera de las 3:

- Red Korb 

- Regina George

- Miranda Priestly

Pero hay un detalle que aun no hemos tomado en cuenta; fue a la sinfonia 3 veces solo en diciembre de 2017

In [76]:
sinfonia = pd.read_sql("""SELECT * FROM facebook_event_checkin
                WHERE date BETWEEN 20171200 AND 20171231
                AND event_name LIKE '%Symphony Concert%' AND person_id IN (78881, 90700, 99716);""", conn)
sinfonia

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


Tres resultados, una persona. Sabemos que el ID de la persona culpable es 99716. Ahora solo hay que ligarlo a su nombre y tenemos a la segunda culpable

### Mente maestra detras de todo

In [86]:
Culpable2 = pd.read_sql("""SELECT name FROM person WHERE id = '99716'""", conn)
Culpable2

,name
0,Miranda Priestly


Ahora sabemos que la culpable detras de todo se llama Miranda Priestly

In [87]:
cursor = conn.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'